In [1]:
import pandas as pd
import numpy as np
import csv as csv
import math
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from sklearn.cross_validation import train_test_split
import time
import time
from datetime import date
import datetime

%matplotlib inline

In [2]:
train_df = pd.read_csv('../hackerrank-predict-email-opens-dataset/training_dataset.csv/training_dataset.csv', header=0)  
train_df.head()

,user_id,mail_id,mail_category,mail_type,sent_time,open_time,click_time,unsubscribe_time,last_online,hacker_created_at,...,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,unsubscribed
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,1463497837,1.463541e+09,NaN,NaN,1.459520e+09,1432533023,...,0,0,0,0,13,0,0,13,0,False
1,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1461357640,NaN,NaN,NaN,1.461210e+09,1432184291,...,0,3,16,0,83,0,43,83,4,False
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1463499639,NaN,NaN,NaN,1.463411e+09,1433045937,...,0,0,0,0,16,0,3,16,0,False
3,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,1463182983,NaN,NaN,NaN,1.462768e+09,1432184291,...,0,0,16,0,85,0,9,85,0,False
4,CYRcuV0cR0algMZJ1N6+3uKcqi8iu+6tJNzmBbmgN7o=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,1461855019,NaN,NaN,NaN,1.461248e+09,1432998058,...,0,1,17,0,43,0,0,43,0,False


Creating bins with Columns
---------------

#### Auto bins creations

In [3]:
train_df = pd.read_csv('../hackerrank-predict-email-opens-dataset/training_dataset.csv/training_dataset.csv', header=0)  

In [4]:
dataTemp  = train_df['submissions_count'].value_counts()
hist, edges = np.histogram(dataTemp, bins=10)
print hist
print edges
dataTempDict = dataTemp.to_dict()
train_df['submissions_count'] = train_df['submissions_count'].map(lambda x: [i for i,v in enumerate(edges) if v<=dataTempDict[x]][-1] )
dummies_submissions_count =  pd.get_dummies(train_df.submissions_count,prefix='submissions_count')
train_df = pd.concat([train_df, dummies_submissions_count], axis=1)
train_df.head()

[1080   12    3    1    0    0    0    0    0    1]
[  1.00000000e+00   6.07270000e+03   1.21444000e+04   1.82161000e+04
   2.42878000e+04   3.03595000e+04   3.64312000e+04   4.25029000e+04
   4.85746000e+04   5.46463000e+04   6.07180000e+04]


,user_id,mail_id,mail_category,mail_type,sent_time,open_time,click_time,unsubscribe_time,last_online,hacker_created_at,...,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,unsubscribed,submissions_count_0,submissions_count_1,submissions_count_2,submissions_count_3,submissions_count_10
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,1463497837,1.463541e+09,NaN,NaN,1.459520e+09,1432533023,...,0,0,13,0,False,0.0,1.0,0.0,0.0,0.0
1,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1461357640,NaN,NaN,NaN,1.461210e+09,1432184291,...,0,43,83,4,False,1.0,0.0,0.0,0.0,0.0
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1463499639,NaN,NaN,NaN,1.463411e+09,1433045937,...,0,3,16,0,False,0.0,1.0,0.0,0.0,0.0
3,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,1463182983,NaN,NaN,NaN,1.462768e+09,1432184291,...,0,9,85,0,False,1.0,0.0,0.0,0.0,0.0
4,CYRcuV0cR0algMZJ1N6+3uKcqi8iu+6tJNzmBbmgN7o=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,1461855019,NaN,NaN,NaN,1.461248e+09,1432998058,...,0,0,43,0,False,1.0,0.0,0.0,0.0,0.0


##### passing the bin edges to the histogram creation


In [5]:
train_df = pd.read_csv('../hackerrank-predict-email-opens-dataset/training_dataset.csv/training_dataset.csv', header=0)  

In [6]:
dataTemp  = train_df['submissions_count'].value_counts()
hist, edges = np.histogram(dataTemp, bins=[1.00000000e+00 ,22768.,46786.8,70805.6,94824.4,
                                           118843.2,142862.,166880.8,190899.6,
                                           214918.4,238937.2,262956.])
print hist
print edges
dataTempDict = dataTemp.to_dict()
train_df['submissions_count'] = train_df['submissions_count'].map(lambda x: [i for i,v in enumerate(edges) if v<=dataTempDict[x]][-1] )
dummies_submissions_count =  pd.get_dummies(train_df.submissions_count,prefix='submissions_count')
train_df = pd.concat([train_df, dummies_submissions_count], axis=1)
train_df.head()

[1095    1    1    0    0    0    0    0    0    0    0]
[  1.00000000e+00   2.27680000e+04   4.67868000e+04   7.08056000e+04
   9.48244000e+04   1.18843200e+05   1.42862000e+05   1.66880800e+05
   1.90899600e+05   2.14918400e+05   2.38937200e+05   2.62956000e+05]


,user_id,mail_id,mail_category,mail_type,sent_time,open_time,click_time,unsubscribe_time,last_online,hacker_created_at,...,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,unsubscribed,submissions_count_0,submissions_count_1,submissions_count_2
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,1463497837,1.463541e+09,NaN,NaN,1.459520e+09,1432533023,...,0,13,0,0,13,0,False,1.0,0.0,0.0
1,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1461357640,NaN,NaN,NaN,1.461210e+09,1432184291,...,0,83,0,43,83,4,False,1.0,0.0,0.0
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1463499639,NaN,NaN,NaN,1.463411e+09,1433045937,...,0,16,0,3,16,0,False,1.0,0.0,0.0
3,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,1463182983,NaN,NaN,NaN,1.462768e+09,1432184291,...,0,85,0,9,85,0,False,1.0,0.0,0.0
4,CYRcuV0cR0algMZJ1N6+3uKcqi8iu+6tJNzmBbmgN7o=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,1461855019,NaN,NaN,NaN,1.461248e+09,1432998058,...,0,43,0,0,43,0,False,1.0,0.0,0.0
